<a href="https://colab.research.google.com/github/Saumi18/Music-Genre-Classification/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload the features_3_sec.csv file below:

In [ ]:
#Upload the features_3_sec.csv file here
from google.colab import files
uploaded = files.upload()

Upload the feature_30_sec.csv file below:

In [ ]:
#Upload the feature_30_sec.csv file here
from google.colab import files
uploaded = files.upload()

- Setting Up & Loading Data
- Feature Scaling & Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("features_3_sec.csv")
df.head()

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=["filename", "length"])

# Encode genre labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])  # Encode genres as numbers

# Separate features (X) and labels (y)
X = df.drop(columns=["label"])
y = df["label"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Model 1 : KNN

In [ ]:
# Model 1: KNN

Model 2: SVM

In [ ]:
# Model 2: SVM

Model 3 : Decision tree/Random Forest

In [ ]:
# Model 3: Decision tree/Random Forest

Model 4: Principal Component Analysis (PCA) + Logistic Regression

In [ ]:
# Model 4: Principal Component Analysis (PCA) + Logistic Regression

Model 5: Gaussian Mixture Model (GMM) + Expectation Maximization

In [ ]:
# Model 5: Gaussian Mixture Model (GMM) + Expectation Maximization



Model 6: Artificial Neural Network (ANN)

In [ ]:
# Model 6: Artificial Neural Network (ANN)

Model 7: Clustering (DBSCAN)

In [ ]:
# Model 7: Clustering DBSCAN